In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

Copy your score files (files ending in .sc) into the direcotry with this notebook. If in binder, upload into binder.

Constraint variables are determined from the initial dataframe generated.

In [ ]:
listdata = []

for i in os.listdir('.'): # Searches current directory
    if i.endswith('.sc'): # Only score files ending in ".sc"
        try:
            d = pd.read_csv(i, sep='\s+')
            listdata.append(d)
        except FileNotFoundError: # This skips missing score files.
            continue
df = pd.concat(listdata) # Creates a dataframe (DF) from score file data

cst_list = []

for i in df: # Searches DF for specific constraint energy terms, this is important for docking!
    if i.endswith('_all_cst'):
        cst_list.append(df[i])
        
for j in cst_list: # This creates a new constraint column for analysis.
    df['Constraint'] = sum(cst_list)

df.sort_values('total_score', ascending=True) # Sorts DF by total score with lowest energy on top.
df # Prints entire dataframe.

The interaction energy variable is determined.

This step truncates the data being presented to only the score, interaction energy, constraint and description (pose name). 

If you are not interested in constraint filters, comment the filter line out (add # at line start).

In [ ]:
for i in df: # Searches DF for specific interface energy term, this is important for docking!
    if i.endswith('_E_1_2'): # Interface energy alwasy ends with "_E_1_2"
        print(i) # Prints the DF name of the interface energy
        interfaceE = i
    else: # Skips all other terms
        pass
# Creates a second Df containting description (pdb), Total Score and interface energy
df_copy = df[['total_score',interfaceE,'Constraint','description']] 
df_copy = df_copy[df_copy.Constraint < 1.00] # This is a constraint filter.
print(str(len(df_copy))) # Prints how many lines are in the dataframe.
df_copy.sort_values('total_score', ascending=True)#[0:10] # Sorts DF by total score with 10 lowest energy.

This next step analyzes docking results from the lowest poses with interaction energy.

In [ ]:
df_copy2 = df_copy
df_copy2.sort_values(interfaceE, ascending=True)[0:10] # Sorts DF by interface energy, shows 10 lowest energy.

In [ ]:
total_mean = round(df.total_score.mean(),1)
inter_mean = round(df[interfaceE].mean(),1)
print("Total Score Average = " + str(total_mean))
print("Lowest Total Score Value = " + str(df.total_score.min()))
print("Interface Energy Average = " + str(inter_mean))
print("Lowest Interface Energy Value = " + str(df[interfaceE].min()))

In [ ]:
df.plot.scatter(x='total_score',y=interfaceE) # Creates a scatter plot
plt.xlabel('Total Score', fontsize='12') # x-axis Label
plt.ylabel('Interface Energy', fontsize='12') # y-axis Label
plt.title('Interface Energy vs. Total Score', fontsize='14') # Plot Title
plt.axvline(total_mean,color='k', linestyle='dashed', linewidth=2) # Creates average for x-axis values
plt.axhline(inter_mean,color='k', linestyle='dashed', linewidth=2) # Creates average for y-axis values
plt.text(df.total_score.max() - 3*df.total_score.std(), 
         df[interfaceE].min(), 
         'Total Score Average = %s\nInterface Energy Average = %s' 
         % (total_mean, inter_mean), 
         fontsize='10') # Text and text placement
#plt.axvline(-1779.7,color='r', linestyle='dashed', linewidth=1.5) #control line
#plt.axhline(6.87,color='r', linestyle='dashed', linewidth=1.5) #control line
#plt.savefig('NAME.png')